# Function Calling with Llama 3.1: Getting started

In this example, we’ll demonstrate how to use function calling with the Llama 3.1 model (70B) using a custom system prompt. We’ll integrate a simple function called calculateTool, which the model will be able to call when appropriate. The function attributes will be defined, and we’ll use the special llama-agentic-system syntax from Meta to pass the function in the system prompt.

In [ ]:
#Install these required dependencies to run this notebook
!pip install openai
!pip install python-dotenv==1.0.0

In [ ]:
# Import libraries
import json
import re
from openai import OpenAI
from dotenv import load_dotenv
import os

# Constants and Configuration
load_dotenv()
API_KEY = os.getenv("SAMBANOVA_API_KEY")
BASE_URL = "https://fast-api.snova.ai/v1"
MODEL = "Meta-Llama-3.1-70B-Instruct"

# OpenAI client setup
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

In [ ]:
# Calculate tool definition
CALCULATE_TOOL = {
    "name": "calculate",
    "description": "Evaluate a mathematical expression",
    "parameters": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "The mathematical expression to evaluate",
            }
        },
        "required": ["expression"],
    },
}

# Tool prompt
TOOL_PROMPT = f"""
You have access to the following functions:

Use the function '{CALCULATE_TOOL["name"]}' to '{CALCULATE_TOOL["description"]}':
{json.dumps(CALCULATE_TOOL)}

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{{"example_name": "example_value"}}</function>

Reminder:
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- If there is no function call available, answer the question like normal with your current knowledge and do not tell the user about function calls
"""


def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})


def parse_tool_response(response: str):
    """Output parser"""
    function_regex = r"<function=(\w+)>(.*?)</function>"
    match = re.search(function_regex, response)

    if match:
        function_name, args_string = match.groups()
        try:
            args = json.loads(args_string)
            return {"function": function_name, "arguments": args}
        except json.JSONDecodeError as error:
            print(f"Error parsing function arguments: {error}")
            return None
    return None


def get_streaming_response(messages):
    response = client.chat.completions.create(
        model=MODEL, messages=messages, max_tokens=1024, temperature=0, stream=True
    )
    return "".join(chunk.choices[0].delta.content or "" for chunk in response)

In [ ]:
def main():
    messages = [
        {"role": "system", "content": TOOL_PROMPT},
        {"role": "user", "content": "What is 25 * 4 + 10?"},
    ]

    # Get initial response
    message = get_streaming_response(messages)
    messages.append({"role": "assistant", "content": message})
    parsed_response = parse_tool_response(message)

    if parsed_response:
        available_functions = {"calculate": calculate}
        function_to_call = available_functions[parsed_response["function"]]
        function_result = function_to_call(parsed_response["arguments"]["expression"])
        messages.append({"role": "ipython", "content": function_result})

        # Get final response
        final_message = get_streaming_response(messages)
        print("Answer from the LLM:", final_message)
    else:
        print("No function call found in the response")


if __name__ == "__main__":
    main()